# PyTorch Adapt を用いた DANN の実装

## 事前準備

- ニューラルネットワークの学習を高速化するためにランタイムを GPU に変更します
- ツールバーから「ランタイム」→「ランタイムのタイプを変更」と進み、「ハードウェアアクセラレータ」をGPUに選択して「保存」を選択しましょう

## ライブラリのインストール

In [6]:
!pip install -q pytorch-adapt

## ライブラリのインポート

In [7]:
# PyTorch 
import torch

# プログレスバーの表示
from tqdm import tqdm

# PyTorch Adapt 関連
from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm
from pytorch_adapt.hooks import DANNHook
from pytorch_adapt.models import Discriminator, mnistC, mnistG
from pytorch_adapt.utils.common_functions import batch_to_device
from pytorch_adapt.validators import IMValidator

## データのダウンロード

In [8]:
# ソース： MNIST、ターゲット： MNIST-M をダウンロード
datasets = get_mnist_mnistm(["mnist"],["mnistm"], folder=".", download=True)

In [9]:
# データローダを作成
dc = DataloaderCreator(batch_size=32)  # バッチサイズ 
dataloaders = dc(**datasets) # ソース・ターゲットなどをまとめたデータセットを作成

## モデルの定義

In [10]:
# デバイス情報を取得
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# 共通の特徴抽出器（畳み込み層2層文）を定義
G = mnistG(pretrained=False).to(device)

# クラス分類のための全結合層3層を定義
C = mnistC(pretrained=False).to(device)

# 敵対的学習のための全結合層2層を定義
D = Discriminator(in_size=1200, # 入力次元数
                  h=256 # 中間層のユニット数
                  ).to(device)

# 3 つをまとめてひとつのモデルとする
models = Models({"G": G,"C": C, "D": D})

# 最適化アルゴリズムとして、Adam　を使用
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.001}))

# モデルのパラメータをオプティマイザに登録
optimizers.create_with(models)
optimizers = list(optimizers.values())

# 損失関数やモデル更新をまとめた Hook を定義
hook = DANNHook(optimizers)

# 評価指標をまとめた Validator を定義
validator = IMValidator()

## 学習の実行

In [ ]:
# エポック数
num_epoch = 2

# 学習と評価を行うループ
for epoch in range(num_epoch):
    # モデルを学習モードに変更
    models.train()
    
    # ソース・ターゲットをまとめたデータローダ train を使って学習
    for data in tqdm(dataloaders["train"]):
        # データとラベルを GPU へ転送
        data = batch_to_device(data, device)
        # パラメータ更新を実行し、損失を取得
        _, loss = hook({**models, **data})

    # モデルを評価モードに変更
    models.eval()

    # クラス分類時の負の対数尤度をまとめるリスト
    logits = []

    # 評価なので、勾配計算のためのメモリ使用は行わずに以下を実行
    with torch.no_grad():
        # 今回はターゲットの学習データを取得
        for data in tqdm(dataloaders["target_train"]):
            # データとラベルを GPU へ転送
            data = batch_to_device(data, device)
            # 特徴抽出器→分類器の順で順伝播して、負の対数尤度を計算
            logits.append(C(G(data["target_imgs"])))
        # 負の対数尤度のリストを更新
        logits = torch.cat(logits, dim=0)

    # ターゲットの負の対数尤度の平均を計算
    score = validator(target_train={"logits": logits})

    # 表示
    print(f"\nEpoch {epoch} score = {score}\n")

100%|██████████| 1844/1844 [01:05<00:00, 28.26it/s]



Epoch 0 score = 1.0237963199615479



 98%|█████████▊| 1806/1844 [01:02<00:01, 35.54it/s]